In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 2: Set Up Paths
import os

# Change this to your specific dataset path in Google Drive
dataset_dir = '/content/drive/MyDrive/new GAN/data'
mask_label_dir = os.path.join(dataset_dir, 'mask label')

# Example output path for processed features
output_dir = '/content/drive/MyDrive/new GAN/output GAN'

In [ ]:
# Step 3: Install Required Libraries
!pip install torch torchvision pillow

In [ ]:
# Step 4: Import Libraries
import json
import os
from PIL import Image, ImageDraw
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Step 5: Define Functions for Feature Extraction
def load_json(json_file_path):
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    return data

def extract_features(json_data, image_path):
    image = Image.open(image_path)
    features = {}

    for shape in json_data['shapes']:
        label = shape['label']
        points = shape['points']
        mask = Image.new('L', (image.width, image.height), 0)
        ImageDraw.Draw(mask).polygon(points, outline=1, fill=1)
        mask = mask.convert('1')
        feature = Image.composite(image, Image.new('RGB', (image.width, image.height)), mask)
        features[label] = feature
    return features

def save_features(features, output_dir, color_label):
    color_output_dir = os.path.join(output_dir, color_label)
    os.makedirs(color_output_dir, exist_ok=True)
    for label, feature in features.items():
        feature.save(os.path.join(color_output_dir, f"{label}.png"))

def process_mask_folders(mask_label_dir, image_base_dir, output_dir):
    # Define the mapping between mask folders and hoodie folders
    color_mapping = {
        'mask black': 'black hoodies',
        'mask brown': 'brown hoodies',
        'mask blue': 'blue hoodies',
        'mask green': 'green hoodies',
        'mask pink': 'pink hoodies',
        'mask white': 'white hoodies'
    }

    # Iterate through each mask color folder
    for mask_color_folder in os.listdir(mask_label_dir):
        mask_color_path = os.path.join(mask_label_dir, mask_color_folder)

        if os.path.isdir(mask_color_path):
            # Find the corresponding hoodie color folder
            hoodie_color_folder = color_mapping.get(mask_color_folder)
            if hoodie_color_folder is None:
                print(f"No matching hoodie folder for {mask_color_folder}")
                continue

            # Full path to the hoodie color folder
            hoodie_color_path = os.path.join(image_base_dir, hoodie_color_folder)

            # Process each JSON file in the mask folder
            for json_file in os.listdir(mask_color_path):
                if json_file.endswith('.json'):
                    json_file_path = os.path.join(mask_color_path, json_file)

                    # Load JSON data
                    json_data = load_json(json_file_path)

                    # Extract the base filename (without the "_mask.json" suffix)
                    base_filename = json_file.replace('_mask.json', '')

                    # Construct the full path to the corresponding hoodie image
                    image_file = f"{base_filename}.jpg"
                    image_path = os.path.join(hoodie_color_path, image_file)

                    if os.path.exists(image_path):
                        # Extract features using the JSON mask and the corresponding image
                        features = extract_features(json_data, image_path)

                        # Save the extracted features
                        save_features(features, output_dir, hoodie_color_folder)
                    else:
                        print(f"Image not found: {image_path}")

In [ ]:
# Step 6: Define DCGAN Architecture
class Generator(nn.Module):
    def __init__(nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Step 7: Train the DCGAN
def train_dcgan(dataloader, netG, netD, criterion, optimizerG, optimizerD, nz, num_epochs=5):
    real_label = 1.
    fake_label = 0.

    for epoch in range(num_epochs):
        for i, data in enumerate(dataloader, 0):
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            # Train with all-real batch
            netD.zero_grad()
            real_cpu = data[0].cuda()
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), real_label, dtype=torch.float, device='cuda')
            output = netD(real_cpu).view(-1)
            errD_real = criterion(output, label)
            errD_real.backward()
            D_x = output.mean().item()

            # Train with all-fake batch
            noise = torch.randn(b_size, nz, 1, 1, device='cuda')
            fake = netG(noise)
            label.fill_(fake_label)
            output = netD(fake.detach()).view(-1)
            errD_fake = criterion(output, label)
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake
            optimizerD.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.fill_(real_label)
            output = netD(fake).view(-1)
            errG = criterion(output, label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optimizerG.step()

            # Print statistics
            if i % 50 == 0:
                print(f'[{epoch}/{num_epochs}][{i}/{len(dataloader)}] Loss_D: {errD.item()} Loss_G: {errG.item()} D(x): {D_x} D(G(z)): {D_G_z1}/{D_G_z2}')

In [ ]:
# Step 8: Generate New Designs
def generate_new_design(netG, nz):
    with torch.no_grad():
        noise = torch.randn(1, nz, 1, 1, device='cuda')
        fake_image = netG(noise).detach().cpu()
    return fake_image

def combine_features(feature_noise_vectors):
    combined_noise = torch.cat(feature_noise_vectors, dim=1)
    return combined_noise

# Example usage
if __name__ == "__main__":
    # Process all mask folders
    process_mask_folders(mask_label_dir, dataset_dir, output_dir)

    # DCGAN parameters
    nz = 100  # Size of z latent vector (i.e., size of generator input)
    ngf = 64  # Size of feature maps in generator
    ndf = 64  # Size of feature maps in discriminator
    nc = 3    # Number of channels in the training images. For color images, this is 3

    # Instantiate the generator and discriminator
    netG = Generator(nz, ngf, nc).cuda()
    netD = Discriminator(nc, ndf).cuda()

    # Loss function and optimizers
    criterion = nn.BCELoss()
    optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

    # Load your dataset (assuming it is in a folder of images)
    from torchvision import datasets, transforms

    dataset = datasets.ImageFolder(root=dataset_dir,
                                   transform=transforms.Compose([
                                       transforms.Resize(64),
                                       transforms.CenterCrop(64),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                   ]))

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

    # Train the DCGAN
    train_dcgan(dataloader, netG, netD, criterion, optimizerG, optimizerD, nz)

    # Generate a new design
    new_design = generate_new_design(netG, nz)
    plt.imshow(np.transpose(new_design[0], (1, 2, 0)))
    plt.show()

    # Example of combining 'sleeve' and 'hood' noise vectors
    noise_sleeve = torch.randn(1, nz // 2, 1, 1, device='cuda')
    noise_hood = torch.randn(1, nz // 2, 1, 1, device='cuda')
    combined_noise = combine_features([noise_sleeve, noise_hood])

    # Generate the combined design
    combined_design = netG(combined_noise)
    plt.imshow(np.transpose(combined_design[0].detach().cpu(), (1, 2, 0)))
    plt.show()

Image not found: /content/drive/MyDrive/new GAN/data/green hoodies/68d26fc8e6f6b62ee6bdc7a9ccd0c934mask.json.jpg
Image not found: /content/drive/MyDrive/new GAN/data/green hoodies/cd01b389ce0ae53fc4a9e4e87e67da6amask.json.jpg
Image not found: /content/drive/MyDrive/new GAN/data/green hoodies/83dea6b537ff47178e25dade28834351mask.json.jpg
Image not found: /content/drive/MyDrive/new GAN/data/green hoodies/34329dae1370e213a39a70dbba7ddc86mask.json.jpg
Image not found: /content/drive/MyDrive/new GAN/data/green hoodies/6857e8fd342cfdc24fd3e06768e46a00mask.json.jpg
Image not found: /content/drive/MyDrive/new GAN/data/green hoodies/5126d6768fbf9a3a9d2177489ae20da3mask.json.jpg
Image not found: /content/drive/MyDrive/new GAN/data/green hoodies/f692bbf9345b997292d842b6dcd9c139mask.json.jpg
Image not found: /content/drive/MyDrive/new GAN/data/green hoodies/097bd3a49c3b8a161b7825535aa95224mask.json.jpg
Image not found: /content/drive/MyDrive/new GAN/data/green hoodies/c5a8377340c8b5ab4ab95f635c13e

TypeError: Generator.__init__() takes 3 positional arguments but 4 were given